In [ ]:
filter_by_fqn: str = ""
filter_by_method_name: str = ""

In [ ]:
import iplantuml
from code_data_science import data_table as dt
import code_data_science.palette as palette

df = dt.read_csv("../samples/call_graph.csv")

In [ ]:
plantuml_code = ["@startuml"]

# TODO: add dynamic coloring based on actions or something in the future
colors = ["black", "red", "blue", "purple"]

classes = {}


def is_valid_class(class_name):
    if filter_by_fqn:
        return (
            True
            if isinstance(class_name, str) and class_name == filter_by_fqn
            else False
        )

    return True


def is_valid_method(method_name):
    if filter_by_method_name:
        return (
            True
            if isinstance(method_name, str) and method_name == filter_by_method_name
            else False
        )

    return True


def make_method(name, arguments):
    # return f"{name}(arguments)" if isinstance(arguments, str) else f"{name}()"
    # overloads are not supported in v1.
    return f"{name}()"


for i, row in df.iterrows():
    from_class = row["fromClass"]
    if is_valid_class(from_class):
        if is_valid_method(row["fromName"]):
            if from_class not in classes:
                classes[from_class] = set()

            members = classes[from_class]
            method_name = make_method(row["fromName"], row["fromArguments"])
            if method_name not in members:
                members.add(make_method(row["fromName"], row["fromArguments"]))

            to_class = row["toClass"]
            if to_class not in classes:
                classes[to_class] = set()

            members = classes[to_class]
            method_name = make_method(row["toName"], row["toArguments"])
            if method_name not in members:
                members.add(make_method(row["toName"], row["toArguments"]))

    to_class = row["toClass"]
    if is_valid_class(to_class):
        if is_valid_method(row["toName"]):
            if to_class not in classes:
                classes[to_class] = set()

            members = classes[to_class]
            method_name = make_method(row["toName"], row["toArguments"])
            if method_name not in members:
                members.add(make_method(row["toName"], row["toArguments"]))

            if from_class not in classes:
                classes[from_class] = set()

            members = classes[from_class]
            method_name = make_method(row["fromName"], row["fromArguments"])
            if method_name not in members:
                members.add(make_method(row["fromName"], row["fromArguments"]))

# Create classes
for _class in classes:
    plantuml_code.append(f"class {_class}")
    plantuml_code.append("{")
    for member in classes[_class]:
        plantuml_code.append(f"    +{member}")
    plantuml_code.append("}")

# Create relationships
for i, row in df.iterrows():
    from_class = row["fromClass"]
    if is_valid_class(from_class):
        if is_valid_method(row["fromName"]):
            from_method = make_method(row["fromName"], row["fromArguments"])
            to_class = row["toClass"]
            to_method = make_method(row["toName"], row["toArguments"])
            # print(f"\"{from_class}::{from_method}\" --> \"{to_class}::{to_method}\"")
            plantuml_code.append(
                f'"{from_class}::{from_method}" --> "{to_class}::{to_method}"'
            )
            # action is not necessary yet.
            # plantuml_code.append(f"\"{from_class}::{from_method}\" --> \"{to_class}::{to_method}\" : {row['action']}")

plantuml_code.append("@enduml")

plantuml_code_string = "\n".join(plantuml_code)

# print(plantuml_code_string)

In [ ]:
cell_content = f"%%plantuml\n{plantuml_code_string}"
get_ipython().run_cell(cell_content)